### Pdf to Txt chunking and no chunking
Prepping data

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load PDF document
loader = PyPDFLoader("CharlesB2.pdf")
data = loader.load()

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)  # Ensure data is in a list if not already

# Assuming all_splits is correctly populated with split documents
with open("buk_all.txt", "w", encoding="utf-8") as text_file:
    for split in all_splits:
        # Check if the unwanted text is in the current split's content
        if "www.PoemHunter.com - The World's Poetry Archive" not in split.page_content:
            text_file.write(split.page_content)  # Write the chunk if it doesn't contain the unwanted text
        else:
            # If the unwanted text is found, replace it with an empty string or perform any other cleaning operation
            cleaned_content = split.page_content.replace("www.PoemHunter.com - The World's Poetry Archive", "")
            text_file.write(cleaned_content)


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load PDF document
loader = PyPDFLoader("Dawn_of_Everything.pdf")
data = loader.load()

with open("DoE_all.txt", "w", encoding="utf-8") as text_file:
    for doc in data:
        text_file.write(doc.page_content + "\n\n")


### Teaching Snippets

In [1]:
# read it in to inspect it
with open('data/buk_all.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
print("length of dataset in characters: ", len(text))
# let's look at the first 1000 characters
print(text[:100])

length of dataset in characters:  99212
2 
16-bit Intel 8088 chip
with an Apple Macintosh
you can't run Radio Shack programs
in its disc dri


In [2]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$&'(),-./0123456789:;<>?ABCDEFGHIJKLMNOPQRSTUVWYabcdefghijklmnopqrstuvwxyzé—
81


In [3]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("kek"))
print(decode(encode("lol?")))

[63, 57, 63]
lol?


In [5]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([99212]) torch.int64
tensor([16,  1,  0, 15, 20, 11, 54, 61, 72,  1, 37, 66, 72, 57, 64,  1, 22, 14,
        22, 22,  1, 55, 60, 61, 68,  0, 75, 61, 72, 60,  1, 53, 66,  1, 29, 68,
        68, 64, 57,  1, 41, 53, 55, 61, 66, 72, 67, 71, 60,  0, 77, 67, 73,  1,
        55, 53, 66,  7, 72,  1, 70, 73, 66,  1, 46, 53, 56, 61, 67,  1, 47, 60,
        53, 55, 63,  1, 68, 70, 67, 59, 70, 53, 65, 71,  0, 61, 66,  1, 61, 72,
        71,  1, 56, 61, 71, 55,  1, 56, 70, 61, 74, 57, 12,  0, 66, 67, 70,  1,
        55, 53, 66,  1, 53,  1, 31, 67, 65, 65, 67, 56, 67, 70, 57,  1, 20, 18,
         0, 56, 70, 61, 74, 57,  1, 70, 57, 53, 56,  1, 53,  1, 58, 61, 64, 57,
         0, 77, 67, 73,  1, 60, 53, 74, 57,  1, 55, 70, 57, 53, 72, 57, 56,  1,
        67, 66,  1, 53, 66,  0, 37, 30, 41,  1, 44, 57, 70, 71, 67, 66, 53, 64,
         1, 31, 67, 65, 68, 73, 72, 57, 70, 12,  0, 54, 67, 72, 60,  1, 39, 53,
        77, 68, 70, 67,  1, 53, 66, 56,  1, 43, 71, 54, 67, 70, 66, 57,  1, 55,
        

In [6]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([16,  1,  0, 15, 20, 11, 54, 61, 72])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

In [8]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[57,  1, 72, 60, 67, 71, 57,  1],
        [58,  1, 67, 66, 57,  1, 67, 58],
        [60, 67, 71, 57,  1, 58, 70, 57],
        [53, 70, 57,  1, 56, 53, 54, 54]])
targets:
torch.Size([4, 8])
tensor([[ 1, 72, 60, 67, 71, 57,  1, 75],
        [ 1, 67, 66, 57,  1, 67, 58,  1],
        [67, 71, 57,  1, 58, 70, 57, 57],
        [70, 57,  1, 56, 53, 54, 54, 64]])
----
when input is [57] the target: 1
when input is [57, 1] the target: 72
when input is [57, 1, 72] the target: 60
when input is [57, 1, 72, 60] the target: 67
when input is [57, 1, 72, 60, 67] the target: 71
when input is [57, 1, 72, 60, 67, 71] the target: 57
when input is [57, 1, 72, 60, 67, 71, 57] the target: 1
when input is [57, 1, 72, 60, 67, 71, 57, 1] the target: 75
when input is [58] the target: 1
when input is [58, 1] the target: 67
when input is [58, 1, 67] the target: 66
when input is [58, 1, 67, 66] the target: 57
when input is [58, 1, 67, 66, 57] the target: 1
when input is [58, 1, 67

In [10]:
print(xb) # our input to the transformer

tensor([[57,  1, 72, 60, 67, 71, 57,  1],
        [58,  1, 67, 66, 57,  1, 67, 58],
        [60, 67, 71, 57,  1, 58, 70, 57],
        [53, 70, 57,  1, 56, 53, 54, 54]])


In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 81])
tensor(5.2911, grad_fn=<NllLossBackward0>)

Tk;JaHQA0K6r8.wy3JoeY)R6bLnOuEqd#"o7UG—L)(3QSM6b ?vSzFr!9:bqFécf :fbEedf4bée/g9:qp0FmP—A3$rw<B950lj'


In [13]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [14]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

4.761895656585693


In [15]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


CG4<I$NNfotgS,3nBu:?y—i!'2w(epCP4yd::&F0Ka ?maqlaéTg/ktj'CC<E;JbA#ULc:hxcF/Q-W,C7"HNIMcSBWoPb3!k!IV!>QN—9Ft—no:cC</K6('S;:—exzn2s,pbnm:IVk(nYMipk.a
mROAyJm?'#IE uP("MRGiQIn!g1diz<a"5QN/o9pgT3ALca",i2ubcé3QkDw6z>cUt —j"&!iCHhMRyH:—u9kS7yA'z8b—P9!lB7D6O)zLfD63!'VNIVs407é&5Ke8s8Fa'g3)O.QM$ei8$u3m
44W',E6iTPQb<u.>p
4Qh>cabv07g08KlazRGWlDUC<?N?L5YqéU;SM24-Gl'OUo9pKg9t:MRoé7TY,A:zxcsTéTl5OL#dc2Int >$s6toVnBKQi8EfN'
tg3MKA0 9Fr.89lE;4$<YQjK6b$vBA8;:dGIGI6g(S59'U?3—#V.'"HhtMac"x
asbzchcn)p$NdtgoQfsg3>é#


In [16]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [17]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [18]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [19]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [20]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [21]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [22]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:

Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
Each example across batch dimension is of course processed completely independently and never "talk" to each other
In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
"self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
"Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [23]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [24]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

### Full Code

In [27]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'mps' # 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('data/DoE_all.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

0.217211 M parameters
step 0: train loss 5.0877, val loss 5.0711
step 100: train loss 2.7304, val loss 2.9814
step 200: train loss 2.5994, val loss 2.8418
step 300: train loss 2.5396, val loss 2.7103
step 400: train loss 2.4711, val loss 2.6340
step 500: train loss 2.4228, val loss 2.5809
step 600: train loss 2.3792, val loss 2.5318
step 700: train loss 2.3200, val loss 2.4655
step 800: train loss 2.2702, val loss 2.4315
step 900: train loss 2.2168, val loss 2.3819
step 1000: train loss 2.1898, val loss 2.3405
step 1100: train loss 2.1615, val loss 2.3137
step 1200: train loss 2.1130, val loss 2.2973
step 1300: train loss 2.0853, val loss 2.2626
step 1400: train loss 2.0716, val loss 2.2513
step 1500: train loss 2.0475, val loss 2.2196
step 1600: train loss 2.0185, val loss 2.1983
step 1700: train loss 1.9954, val loss 2.1880
step 1800: train loss 1.9825, val loss 2.1610
step 1900: train loss 1.9577, val loss 2.1578
step 2000: train loss 1.9442, val loss 2.1327
step 2100: train loss 1.

In [28]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


It existerapical societies agained in
the temble state oring
banded for at Lesociate andshes for the Dreamage breason sometimes to leigity evences acclose of
suchtingly Coanistrated the fect if wands trading role have publishouses, the of intennerally as to migcts only ]usarch many . Are to
recocological dislations within; it’ s s
crite freet oraighic and Jack, somerher
stecond
historic epileto-state lobsomuinative and so some that time, ese femonel; sites an in the acceptic; alwhed the formal cless ocies becominal societies
ﬁrdindary difpicte senfrourced outservacilists’.45 In tisto are concents this cain (if freelood don ﬁrst
niving itaso lar ge is that ave of uper kind all soe ofthos fov were dif then, are something as prence, aceors of foragizic showseleded, in the wooging that is than peops’ J0000 Jäends ( to be
scond of European steptrued weremedicated.
Hawlds are Archanined at in
Jatheles warrang rangem
teconould at matered from ‘or anireatones – they women Canque clear , enso 